In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
train_df = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv',index_col='id')
test_df = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv',index_col='id')

# Explanatory Data Analysis

In [ ]:
train_df

In [ ]:
#change date datatype as datetime
train_df.date = pd.to_datetime(train_df.date)
test_df.date= pd.to_datetime(test_df.date)

train_df['year'] = train_df.date.dt.year
test_df['year'] = test_df.date.dt.year

train_df['month'] = train_df.date.dt.month
test_df['month'] = test_df.date.dt.month

train_df['dayofmonth'] = train_df.date.dt.day
test_df['dayofmonth'] = test_df.date.dt.day

train_df['dayofweek'] = train_df.date.dt.dayofweek
test_df['dayofweek'] = test_df.date.dt.dayofweek

train_df['dayname'] = train_df.date.dt.strftime('%A')
test_df['dayname'] = test_df.date.dt.strftime('%A')

<font size=4>In this competition, need to predict the sales of stores the last two weeks of August 2017. <font>

In [ ]:
print('Train: ',min(train_df.date),max(train_df.date))
print('\n')
print('Test: ',min(test_df.date),max(test_df.date))

In [ ]:
#store number from 1 -54
#sorted(list(train_df.store_nbr.unique()))


<font size=4>Onpromotion gives the total number of items in a product family that were being promoted at a store at a given date. <font>

In [ ]:
train_df.onpromotion.unique()

t

In [ ]:
train_df.family.unique()

# Yearly Growth  by Product Family
* <font size=4> Almost all product families were growing stably to 2017.</font>

* <font size=4> There are some exceptions for the tendency. Frozen foods,baby care, and liquor,beer, wine sales decreased from 2016. They might get affected by the Earthquake in Equador on April 16, 2016.</font>

In [ ]:
fig= plt.figure(figsize=(10,120))
fig.subplots_adjust(hspace=0.75)
for i,product in enumerate(train_df.family.unique()):
    ax = fig.add_subplot(33,1,i+1)
    train_df.query('family==@product').groupby(['year']).sales.mean().plot()
    plt.title(product);



# Monthly Sales by Year
* <font size=4> It is hard to see some seasonal effects from the graphs except for some items of the christmas season.Frozen foods are obviously affected by the Christmas season.</font>

* <font size=4> Books sold well in 2016 from septemper to the end of the year. </font>


In [ ]:
fig= plt.figure(figsize=(10,120))
fig.subplots_adjust(hspace=0.75)
for i,product in enumerate(train_df.family.unique()):
    ax = fig.add_subplot(33,1,i+1)
    select = train_df.query('family==@product')
    for year in [2013,2014,2015,2016,2017]: 
        select.query('year==@year').groupby('month').sales.mean().plot(ax=ax,label=year)
    plt.title(product)
    ax.legend();

********************************************************************
<font size=4> No missing values in the dataset. </font>

In [ ]:
print(train_df.isna().sum())
print(test_df.isna().sum())

# Puplic sector's Paid Day affects the sales?

* <font size=4> "Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this" from the additional notes on data page.</font>

* <font size=4> Not so obvious but theare are two sales peaks around the paydays. </font>

In [ ]:
train_df.groupby('dayofmonth').sales.mean().plot(kind='bar')
plt.title('Sales Average by Day of Month')

<font size=4>More people go to grocery on weekends.</font>

In [ ]:
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
train_df.groupby('dayname').sales.mean().reindex(index=order).plot(kind='bar')
plt.title('Average Sales by Day of week')

# Sales Ranking

<font size=4> Sales is depending on the location,store size, promotions, and so on. We can check those factors later with the data from the other file.

In [ ]:
plt.figure(figsize=(8,8))
train_df.groupby('store_nbr').sales.sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Sales Ranking by Store')


In [ ]:
plt.figure(figsize=(8,8))
train_df.groupby('store_nbr').onpromotion.sum().sort_values(ascending=False).plot(kind='bar',color='red')
plt.title('Promotion Items Ranking by Store')

* <font size=4> Promotion and sales have positive corellation as the graph shows. Store number 53 has many promotions but does not have much sales.</font>

In [ ]:
train_df.groupby('store_nbr')[['onpromotion','sales']].sum().plot.scatter('onpromotion','sales')
plt.title('Promotion and Sales Relationship')

# Oil CSV

* <font size=4>According to the additional notes on the data page, Equador is a n oil-dependent country. Oil price is econimical health. There might be two oil shocks from 2014 to 2016</font>

* <font size=4>Adding GDP data will help predictions.</font>

In [ ]:
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv',index_col='date')
oil.plot();
plt.xticks(rotation=45);

In [ ]:
#filling missing data
oil = oil.interpolate(method='linear')
#the first row is still missing the value
oil.iloc[0] = oil.iloc[1]
oil.plot();
plt.xticks(rotation=45);

In [ ]:
import datetime

#some days are skipped. Filling up the gap.

start_date = train_df.date.min() 
# from beggining of the train date and the end of test date
number_of_days = 1704 #1703
date_list = [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days)]

date = (pd.Series(date_list)).to_frame()
date.columns = ['date']
date.date = pd.to_datetime(date.date)
date['date_str'] = date.date.astype(str)
oil['date_str'] = oil.index.astype(str)

oil = pd.merge(date,oil,how='left',on='date_str')



In [ ]:
oil.set_index('date').dcoilwtico.plot()


In [ ]:
oil = oil.set_index('date').dcoilwtico.interpolate(method='linear').to_frame()
#oil.set_index('date').dcoilwtico.plot();



In [ ]:
oil['date_str'] = oil.index.astype(str)


In [ ]:
# add oil price to the train date
train_df['date_str'] = train_df.date.astype(str)

train_df = pd.merge(train_df,oil,how='left',on='date_str')
train_df.drop(columns='date_str',axis=1,inplace=True)


In [ ]:
test_df['date_str'] = test_df.date.astype(str)
test_df = pd.merge(test_df,oil,how='left', on='date_str')

test_df.drop(columns='date_str',axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
train_df.groupby('family')[['sales','dcoilwtico']].corr().iloc[0::2,-1].sort_values()


<font size =4> Some product family (Home Care,Magazines,Home and Kitchen II,Produce,Home and Kitchen I)<br>
    have slightly stronger correlationship with the oil price.
    </font>

# Stores CSV

In [ ]:
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv',index_col='store_nbr')
train_df = pd.merge(train_df,stores,how='left',on='store_nbr')
test_df = pd.merge(test_df,stores,how='left',on='store_nbr')

<font size=4> Type A looks like a biggest type of store and Type C is the smalles size of store.
    </font>

In [ ]:
train_df.groupby(['type']).sales.mean().plot(kind='bar');

<font size = 4> There will be positive corellation with the number of stores and the city population. Guayaquil is the most populous city in 2022, and Quito is the second most populous city.
    </font>

In [ ]:
plt.figure(figsize=(10,4))

ax1 = plt.subplot(1,2,1)
train_df.groupby(['city']).sales.mean().plot(kind='bar')
plt.title('Average Sales by City')
ax2 = plt.subplot(1,2,2)
train_df.groupby(['city'])['store_nbr'].nunique().plot(kind='bar')
plt.title('Number of Stores by City')

<font size=4>According to the data description, the cluster is a grouping of similar stores. I cannot identify how the cluster and the type is different from the dataset.
</font>

In [ ]:
train_df.groupby(['cluster']).sales.mean().plot(kind='bar')

In [ ]:
train_df.groupby(['cluster','store_nbr','city','type']).nunique()

# Holidays_Events.CSV

In [ ]:
holiday_df = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
holiday_df.query('transferred==True')

# transferred day is not celebrated
holiday_df = holiday_df.query('transferred ==False')
holiday_df.description = holiday_df.description.str.replace('Traslado ','')

#national
national = holiday_df.query('locale=="National"')

day_off = national.query('type!="Work Day" or type!="Event"').set_index('date')['description'].to_dict()

train_df['date_str'] = train_df.date.astype(str)
test_df['date_str'] = test_df.date.astype(str)

train_df['national_holiday'] = [1 if a in day_off else 0 for a in train_df.date_str]
test_df['national_holiday'] = [1 if a in day_off else 0 for a in test_df.date_str]

event = national.query('type=="Event"').set_index('date')['description'].to_dict()

train_df['national_event'] =[1 if a in event else 0 for a in train_df.date_str]
test_df['national_event'] =[1 if a in event else 0 for a in test_df.date_str]

work_day = national.query('type=="Work Day"').set_index('date')['description'].to_dict()
train_df['national_workday'] = [1 if a in work_day else 0 for a in train_df.date_str]
test_df['national_workday'] = [1 if a in work_day else 0 for a in test_df.date_str]

#weekend
train_df['weekend'] = [1 if a>=5 else 0 for a in train_df.dayofweek]
test_df['weekend'] = [1 if a>=5 else 0 for a in test_df.dayofweek]

In [ ]:
train_df.columns

In [ ]:
#locale
#locale is corresponding to city of train_df
local = holiday_df.query('locale=="Local"')
local_dic = local.set_index('date').locale_name.to_dict()
train_df['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(train_df.city,train_df.date_str)]
test_df['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(test_df.city,test_df.date_str)]

In [ ]:
#Regional
#Regional is corresonding to state of train_df
regional = holiday_df.query('locale=="Regional"')
regional_dic = regional.set_index('date').locale_name.to_dict()
train_df['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(train_df.state,train_df.date_str)]
test_df['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(test_df.state,test_df.date_str)]



# Transaction.CSV 

In [ ]:
transaction_df = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
fig= plt.figure(figsize=(30,120))
fig.subplots_adjust(hspace=0.75)
for i,store in enumerate(range(1,54)):
    ax = fig.add_subplot(14,4,i+1)
    transaction_df.set_index('date').query('store_nbr==@store').transactions.plot(c='purple');
    plt.title('Store Number: ' + str(store));
    plt.ylabel('Number of Transactions')
    plt.xticks(rotation=45);

<fond size=4>
    Every store have peak days in Christmas season. Some stores show upper trends.

# Preparing Data

In [ ]:
len(train_df.query('date_str=="2013-01-01"'))

In [ ]:
#https://www.kaggle.com/c/store-sales-time-series-forecasting/discussion/291165
# idea and codes comes from this discussion 

train_df.sales = np.log1p(train_df.sales)


In [ ]:
train_df['Istest'] = False
test_df['Istest'] = True

full = pd.concat((train_df,test_df))

#remove leap year day
#full = full.query('date_str !="2016-02-29"')


full['Lag_16'] = full['sales'].shift(1782*16)
full['Lag_17'] = full['sales'].shift(1782*17)
full['Lag_18'] = full['sales'].shift(1782*18)
full['Lag_19'] = full['sales'].shift(1782*19)
full['Lag_20'] = full['sales'].shift(1782*20)
full["Lag_21"] = full['sales'].shift(1782*21)
full['Lag_22'] = full['sales'].shift(1782*22)
full['Lag_28'] = full['sales'].shift(1782*28)
full['Lag_31'] = full['sales'].shift(1782*31)

full['Lag_365'] = full['sales'].shift(1782*365)


train_df = full.query('Istest==False')
test_df = full.query('Istest ==True')


In [ ]:
train_df = train_df.dropna(subset=['Lag_365'],axis=0)

In [ ]:
FEATURES = ['store_nbr','family','onpromotion', 'year', 'month',
       'dayofmonth', 'dayofweek','dcoilwtico', 'city', 'state',
       'type', 'cluster', 'national_holiday', 'national_event',
       'national_workday', 'weekend', 'local_holiday', 'regional_holiday','Lag_16','Lag_17','Lag_18','Lag_19','Lag_20','Lag_21','Lag_22','Lag_28','Lag_31','Lag_365']
TARGET =['sales']



In [ ]:
from sklearn import preprocessing
categories = ['family','city','state','type']
for i in categories:
    encoder = preprocessing.LabelEncoder()
    train_df[i] = encoder.fit_transform(train_df[i])
    test_df[i] =  encoder.transform(test_df[i])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_df,train_df[TARGET],test_size=0.05,shuffle=False)

In [ ]:
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_log_error

linear = LinearRegression()
model = linear.fit(X_train[FEATURES],y_train)


#plt.scatter(X_val.date,model.predict(X_val[FEATURES]))
#plt.xticks(rotation=45)

predictions= model.predict(X_val[FEATURES])
predictions = [a if a>0 else 0 for a in predictions]
print('MSLE: ' + str(mean_squared_log_error(y_val,predictions)))


In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(tree_method='gpu_hist',n_estimators=500)
xgb.fit(X_train[FEATURES], y_train,
        eval_set=[(X_train[FEATURES],y_train),(X_val[FEATURES], y_val)],
       verbose=False,early_stopping_rounds=10)

#plt.scatter(X_val.date,xgb.predict(X_val[FEATURES]))
#plt.xticks(rotation=45)

#predictions have negative. Changed negative values to 0.
predictions= xgb.predict(X_val[FEATURES])
predictions = [a if a>0 else 0 for a in predictions]
print('MSLE: ',mean_squared_log_error(y_val,predictions))

In [ ]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor()
cat.fit(X_train[FEATURES], y_train,
        eval_set=[(X_train[FEATURES],y_train),(X_val[FEATURES], y_val)],
       verbose=False,early_stopping_rounds=10)


#predictions have negative. Changed negative values to 0.
cat_predictions= cat.predict(X_val[FEATURES])
cat_predictions = [a if a>0 else 0 for a in cat_predictions]
print('MSLE: ',mean_squared_log_error(y_val,cat_predictions))

In [ ]:
from lightgbm import LGBMRegressor

lg = LGBMRegressor()
lg.fit(X_train[FEATURES], y_train,
        eval_set=[(X_train[FEATURES],y_train),(X_val[FEATURES], y_val)],
       verbose=False,early_stopping_rounds=10)


#predictions have negative. Changed negative values to 0.
lg_predictions= lg.predict(X_val[FEATURES])
lg_predictions = [a if a>0 else 0 for a in lg_predictions]
print('MSLE: ',mean_squared_log_error(y_val,lg_predictions))

In [ ]:
def objective(trial,X,y, name='xgb'):
    params = param = {
        'tree_method':'gpu_hist',  
        #'eval_metrics': 'rmse',
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'eta': trial.suggest_float('eta', 1e-5, 0.1),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical( 
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.009,0.01,0.012,0.016, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300,500,1000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
        'random_state':10
        }

    model =  XGBRegressor(**params)
    model.fit(X_train[FEATURES],y_train,eval_set=[(X_val[FEATURES],y_val)],early_stopping_rounds=50,verbose=False)

    predictions_train = [a if a>0 else 0 for a in model.predict(X_train[FEATURES])]
    predictions_val  = [a if a>0 else 0 for a in model.predict(X_val[FEATURES])]           
    
    train_score = np.round(mean_squared_log_error(y_train, predictions_train), 5)
    test_score = np.round(mean_squared_log_error(y_val, predictions_val), 5)
                  
    print(f'TRAIN MSLE : {train_score} || TEST MSLE : {test_score}')
                  
    return test_score

In [ ]:
#%%time

#from functools import partial
#import optuna

#optimize = partial(objective,X=X_train,y=y_train)

#study_lgbm = optuna.create_study(direction ='minimize')
#study_lgbm.optimize(optimize,n_trials=50)

In [ ]:
params = {'lambda': 6.105970537016599, 'alpha': 0.874716179324655, 'eta': 0.047228549789593455, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.012, 'n_estimators': 1000, 'max_depth': 17, 'min_child_weight': 155}

# Making a submission file

In [ ]:
xgb = XGBRegressor(tree_method='gpu_hist',**params)
xgb.fit(train_df[FEATURES], train_df[TARGET],verbose=False)

xgb_predictions= xgb.predict(test_df[FEATURES])
xgb_predictions = [a if a>0 else 0 for a in xgb_predictions]

cat_predictions = cat.predict((test_df[FEATURES]))
cat_predictions = [a if a>0 else 0 for a in cat_predictions]

lg_predictions= lg.predict(test_df[FEATURES])
lg_predictions = [a if a>0 else 0 for a in lg_predictions]

In [ ]:
predictions = [0.6* a + 0.3 * b + 0.1 * c for a,b,c in zip(xgb_predictions,cat_predictions,lg_predictions)] 

In [ ]:
output = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv',index_col='id')
output['sales']= np.expm1(predictions)

In [ ]:
output.to_csv('submission.csv')

# References
Referred how to make graphs: [Kaggle merchandise EDA with baseline linear model](https://www.kaggle.com/lucamassaron/kaggle-merchandise-eda-with-baseline-linear-model)

Time series course on Kaggle https://www.kaggle.com/learn/time-series



# To be continued.
<font size=4> I will work on modeling part more. </font>